In [171]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string
from fim import fpgrowth, apriori
import spacy
from spacy.lang.en import stop_words
nlp = spacy.load('en_core_web_sm')
from ortools.linear_solver import pywraplp

In [172]:
data = {
    "review": [
        "The acting was incredible and the story was engaging.",
        "Terrible plot and boring characters ruined the movie.",
        "Amazing cinematography and a powerful message.",
        "The story was confusing and the acting felt fake.",
        "Outstanding performances and a beautiful soundtrack.",
        "Weak story with predictable twists and flat acting.",
        "The direction was brilliant and the characters were deep.",
        "Poor editing and bad dialogue made it hard to watch.",
        "It was a visually stunning film with great acting.",
        "Nothing made sense and the plot was a mess.",
        "The characters were likable and the story flowed well.",
        "Boring pacing and emotionless performances throughout.",
        "A touching story and breathtaking cinematography.",
        "The movie lacked depth and was hard to follow.",
        "Great story with strong performances from the lead actors.",
        "Terrible direction and overused clichés ruined it.",
        "Solid plot, good pacing, and relatable characters.",
        "I hated the acting and the story was awful.",
        "Excellent script and engaging emotional arc.",
        "The film was empty and the characters were annoying.",
        "Good acting but the plot was weak.",
        "The story was heartfelt and the cast was amazing.",
        "Flat performances and boring script.",
        "Wonderful use of visuals and a solid plot.",
        "Nothing new, just a generic story and poor acting.",
        "Emotional journey with powerful performances.",
        "The characters lacked depth and purpose.",
        "Strong direction and great chemistry between leads.",
        "The dialogue was unrealistic and the story forgettable.",
        "A meaningful plot and stunning cinematography.",
        "Terribly written characters and weak direction.",
        "Great visuals and a fast-paced plot.",
        "Boring from start to finish with no real climax.",
        "Amazing acting saved the otherwise dull story.",
        "The plot went nowhere and the film felt lazy.",
        "Cinematic beauty with a compelling story.",
        "Forgettable characters and forced drama.",
        "Superb acting and flawless direction.",
        "The movie was repetitive and dragged on.",
        "Emotional depth and a well-crafted plot.",
        "The acting felt forced and the story was bad.",
        "Powerful message and top-tier performances.",
        "The movie was disjointed and poorly edited.",
        "Beautiful visuals and strong emotional pull.",
        "Awkward pacing and bland characters.",
        "Loved the story and the acting was great.",
        "Overacted and had no coherent story.",
        "Charming characters and a wonderful plot.",
        "Bad acting and an even worse story.",
        "The plot had depth and the characters felt real.",
        "I was bored by the slow plot and dull acting.",
        "It had heart, humor, and great writing.",
        "The film lacked creativity and charm.",
        "Gripping story with excellent direction.",
        "Predictable story and uninspired acting.",
        "The acting was real and the emotions hit hard.",
        "Flat characters and a recycled plot.",
        "Stunning performances and a rich story.",
        "The film was shallow and the acting was bad.",
        "A compelling story and thoughtful writing.",
        "The movie felt empty and forgettable.",
        "Sharp writing and nuanced performances.",
        "The story was weak and full of holes.",
        "Visually beautiful and emotionally powerful.",
        "The characters were dull and uninteresting.",
        "Excellent performances and a meaningful arc.",
        "Bad script and poor production quality.",
        "Great pacing, solid story, and heartfelt moments.",
        "The film lacked focus and dragged.",
        "The plot was tight and emotionally resonant.",
        "Overly long and lacking in substance.",
        "A story worth telling with actors who delivered.",
        "The characters didn’t evolve and the plot stalled.",
        "It had beautiful direction and strong acting.",
        "The story was derivative and boring.",
        "Heartfelt storytelling with emotional depth.",
        "Uninspired dialogue and bland acting.",
        "The film delivered powerful performances and message.",
        "The acting felt wooden and the story had no soul.",
        "A strong cast and a solid emotional journey.",
        "Forgettable plot and messy storytelling.",
        "The characters felt authentic and the story worked.",
        "It was boring and the acting was weak.",
        "Rich storytelling and great performances.",
        "It lacked any memorable moments or performances.",
        "Masterful direction and impactful acting.",
        "No structure, bad pacing, and forgettable characters.",
        "Beautiful cinematography with a solid emotional base.",
        "The film felt rushed and poorly planned.",
        "The acting was grounded and the story was layered.",
        "Pointless scenes and random story arcs.",
        "A great ensemble and a touching story.",
        "Unconvincing acting and flat storytelling.",
        "Incredible acting and a compelling plot.",
        "No chemistry and a poorly written story.",
        "Smart writing and emotionally strong performances.",
        "The film was a mess with no direction.",
        "Believable characters and a powerful narrative.",
        "Disjointed plot and stiff acting.",
        "Touching story and excellent character arcs.",
    ]
}

# df = pd.DataFrame(data)
df = pd.read_csv('YoutubeCommentsDataSet.csv')
print(df.shape)
df.head(100)

(18408, 2)


,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive
...,...,...
95,poison support is one of the more interesting ...,positive
96,my son had botox injections in his legs for ye...,negative
97,my mom wouldve gone nuts for you while i was g...,negative
98,i love how she says that it is poison in the r...,positive


In [173]:
df = df.head(1000)
df.drop(columns=['Sentiment'], inplace=True)
df.columns = ['review']

In [174]:
def tokenize(text):
    if not isinstance(text, str):
        return []
    tokens = text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    return list(set([w for w in tokens if w not in stop_words.STOP_WORDS and w.isalpha()]))

def preprocess(df):
    df.dropna(inplace=True)
    df["transaction"] = df["review"].apply(tokenize)
    transactions = df["transaction"].tolist()
    return transactions

In [175]:
transactions = preprocess(df)
for i, t in enumerate(transactions):
    print(f"{i+1}: {t}")

1: ['order', 'apples', 'brand', 'base', 'wanted', 'successive', 'able', 'older', 'apple', 'iphones', 'iphone', 'portion', 'wasnt', 'technology', 'replaced', 'increased', 'people', 'significant', 'new', 'number', 'use', 'incorporated', 'pay', 'forget', 'required', 'user', 'lets']
2: ['nz', 'apple', 'like', 'contactless', 'support', 'come', 'paywave', 'pay', 'retailers', 'high', 'fees', 'machines', 'credit', 'card']
3: ['lessons', 'explanations', 'helpful', 'youll', 'account', 'help', 'sit', 'monitor', 'comfort', 'growth', 'forever', 'acknowledge', 'ideas', 'channel']
4: ['getting', 'gotten', 'like', 'inconvenient', 'brainwashed', 'carry', 'apple', 'place', 'annoyed', 'feels', 'store', 'habits', 'quickly', 'physical', 'pocket', 'seriously', 'drag', 'happen', 'shock', 'crazy', 'shopping', 'took', 'pay', 'routine', 'officially', 'card']
5: ['secure', 'apple', 'airports', 'use', 'korean', 'cards', 'convenient', 'pay', 'physical', 'japanese', 'credit', 'easy', 'need']
6: ['apple', 'hounding'

In [176]:
# Itemsets frecuentes con soporte mínimo de 2
r = fpgrowth(transactions, supp=5)
df = pd.DataFrame(r)
df.columns = ['Itemset', 'Support']

df.sort_values(by='Support', ascending=False, inplace=True)
df

,Itemset,Support
0,"(like,)",109
2,"(love,)",103
1,"(people,)",80
3,"(video,)",64
7,"(tech,)",60
6,"(best,)",59
5,"(time,)",58
8,"(great,)",58
4,"(good,)",50


In [177]:
r = fpgrowth(transactions, target='r', supp=1, report='aSc')
df = pd.DataFrame(r)
df.columns = ['Consequent', 'Antecedent', 'Freq', 'Freq(%)', 'Conf']
df.sort_values(by='Conf', ascending=False, inplace=True)
df

,Consequent,Antecedent,Freq,Freq(%),Conf
1089,polite,"(achieves,)",10,1.003009,1.0
1088,polite,"(achieves, dude)",10,1.003009,1.0
31,credit,"(takes, videos, best)",11,1.103310,1.0
30,takes,"(credit, videos, best)",11,1.103310,1.0
28,best,"(credit, takes, videos)",11,1.103310,1.0
...,...,...,...,...,...
1101,audience,"(achieves, dude, credit, takes, best)",8,0.802407,0.8
1102,audience,"(achieves, dude, credit, takes)",8,0.802407,0.8
1103,audience,"(achieves, dude, credit)",8,0.802407,0.8
1120,audience,"(achieves, polite)",8,0.802407,0.8


In [178]:
# Convertir las reglas a estructura manejable
rules = fpgrowth(transactions, target='r', supp=1, report='aSc')
rule_data = [
    {"ante": r[0], "cons": r[1], "support": r[2], "conf": r[3]}
    for r in rules if r[3] >= 0.5  # opcional: filtrar confianza mínima
]

solver = pywraplp.Solver.CreateSolver("SCIP")
x = [solver.BoolVar(f"x_{i}") for i in range(len(rule_data))]

# Maximizar la suma de la confianza de las reglas seleccionadas
solver.Maximize(solver.Sum(rule_data[i]["conf"] * x[i] for i in range(len(rule_data))))

# Restringimos a seleccionar máximo 5 reglas
solver.Add(solver.Sum(x) <= 10)

# Resolver
status = solver.Solve()

# Mostrar reglas seleccionadas
if status == pywraplp.Solver.OPTIMAL:
    print("=== Selected rules ===")
    for i in range(len(rule_data)):
        if x[i].solution_value() > 0:
            r = rule_data[i]
            print(f"{r['ante']} => {r['cons']} | Support: {r['support']}%, Confidence: {r['conf']}")
else:
    print("No optimal solution found.")

=== Selected rules ===
polite => ('dude', 'credit', 'takes', 'best') | Support: 13%, Confidence: 1.3039117352056169
takes => ('polite', 'dude', 'credit') | Support: 13%, Confidence: 1.3039117352056169
credit => ('polite', 'dude', 'takes') | Support: 13%, Confidence: 1.3039117352056169
dude => ('polite', 'credit', 'takes') | Support: 13%, Confidence: 1.3039117352056169
polite => ('dude', 'credit', 'takes') | Support: 13%, Confidence: 1.3039117352056169
credit => ('polite', 'dude') | Support: 13%, Confidence: 1.3039117352056169
dude => ('polite', 'credit') | Support: 13%, Confidence: 1.3039117352056169
polite => ('dude', 'credit') | Support: 13%, Confidence: 1.3039117352056169
dude => ('polite',) | Support: 13%, Confidence: 1.3039117352056169
polite => ('dude',) | Support: 13%, Confidence: 1.3039117352056169
